In [1]:
# Copy/Paste the contents to a new file demo.py
import torch
from ipex_llm.transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, GenerationConfig
generation_config = GenerationConfig(use_cache=True)

print('Now start loading Tokenizer and optimizing Model...')
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-1.5B-Instruct",
                                          trust_remote_code=True)

# Load Model using ipex-llm and load it to GPU
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-1.5B-Instruct",
                                             load_in_4bit=True,
                                             cpu_embedding=True,
                                             low_cpu_mem_usage=False,
                                             trust_remote_code=True)
model = model.to('xpu')
print('Successfully loaded Tokenizer and optimized Model!')

# Format the prompt
# you could tune the prompt based on your own model,
# here the prompt tuning refers to https://huggingface.co/Qwen/Qwen2-1.5B-Instruct#quickstart
question = "What is AI?"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": question}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# Generate predicted tokens
with torch.inference_mode():
   input_ids = tokenizer.encode(text, return_tensors="pt").to('xpu')

   print('--------------------------------------Note-----------------------------------------')
   print('| For the first time that each model runs on Intel iGPU/Intel Arc™ A300-Series or |')
   print('| Pro A60, it may take several minutes for GPU kernels to compile and initialize. |')
   print('| Please be patient until it finishes warm-up...                                  |')
   print('-----------------------------------------------------------------------------------')

   # To achieve optimal and consistent performance, we recommend a one-time warm-up by running `model.generate(...)` an additional time before starting your actual generation tasks.
   # If you're developing an application, you can incorporate this warm-up step into start-up or loading routine to enhance the user experience.
   output = model.generate(input_ids,
                           do_sample=False,
                           max_new_tokens=32,
                           generation_config=generation_config) # warm-up

   print('Successfully finished warm-up, now start generation...')

   output = model.generate(input_ids,
                           do_sample=False,
                           max_new_tokens=32,
                           generation_config=generation_config).cpu()
   output_str = tokenizer.decode(output[0], skip_special_tokens=False)
   print(output_str)

c:\Users\84866\miniforge3\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\84866\miniforge3\envs\llm\Lib\site-packages\transformers\deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
c:\Users\84866\miniforge3\envs\llm\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\84866\miniforge3\envs\llm\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environ

Now start loading Tokenizer and optimizing Model...


c:\Users\84866\miniforge3\envs\llm\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2025-02-04 10:31:29,902 - INFO - Converting the current model to sym_int4 format......
c:\Users\84866\miniforge3\envs\llm\Lib\site-packages\torch\nn\init.py:412: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Successfully loaded Tokenizer and optimized Model!
--------------------------------------Note-----------------------------------------
| For the first time that each model runs on Intel iGPU/Intel Arc™ A300-Series or |
| Pro A60, it may take several minutes for GPU kernels to compile and initialize. |
| Please be patient until it finishes warm-up...                                  |
-----------------------------------------------------------------------------------
Successfully finished warm-up, now start generation...
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is AI?<|im_end|>
<|im_start|>assistant
Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and act like humans. It involves the development of computer systems
